In [8]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import re
import time
import json
import datetime
import unicodedata

In [9]:
url_general = 'https://maldita.es/malditobulo/1'
html_general = requests.get(url_general).text
soup_general = BeautifulSoup(html_general, 'html.parser')

In [3]:
clase_nav_bar = 'bg-white text-gray-800 px-3 py-2 text-lg border-gray-400 font-bold'

pags_link = soup_general.find('span', attrs={'class' : clase_nav_bar})('a')[0].get('href', None)
num_pags = int(pags_link.split('/')[-1])
print(num_pags)

565


In [4]:
def replace_characters(text):
    
    text_aux = ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^\w\s]', '', text_aux.lower().replace(u'\xa0', u' ').replace(u'\n', u' ')).split(" ")

In [5]:
bulos = {}

clase_new = 'section-card-headline'
cont = 0
for i in range(1, num_pags+1):
    url = 'https://maldita.es/malditobulo/'+str(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    for div in soup.find_all('div',attrs={'class' : clase_new}):
        try:
            a = div('a')[0]
            title = a.text
            link = a.get('href', None)
            words = dict()
            
            # Obtener la fecha de la noticia
            if "https://maldita.es/malditobulo/" in link:
                date = link.replace("https://maldita.es/malditobulo/", "")[0:8]
            elif "https://migracion.maldita.es/" in link:
                date = link.replace("https://migracion.maldita.es/", "")[0:8]
            else:
                date = link.replace("https://alimentacion.maldita.es/mitos-bulos/", "")[0:8]
            
            formated_date =  datetime.datetime.strptime(date, '%Y%m%d').strftime('%d/%m/%Y')
            
            html_new = requests.get(link, allow_redirects=False).text

            soup_new = BeautifulSoup(html_new, 'html.parser')

            title_list = replace_characters(title.lower())


            for w in title_list:
                if w != '':
                    if w in words:
                        words[w] += 1
                    else:
                        words[w] = 1

                    
            new_text = soup_new.find('div',attrs={'id' : "article-text"})            
            
            text_list = new_text('p') + new_text('li')

            
            for text in text_list:
                if(not text.get_text().startswith("Una publicación compartida de")):
                    for w in replace_characters(text.get_text()):
                        if w != '':
                            if w in words:
                                words[w] += 1
                            else:
                                words[w] = 1


            bulos[cont] = {"titulo" : title, "link" : link, "date" : formated_date, "words_count" : words}
        except:
            print("No se ha podido obtener la informacion de la noticia", i)
        cont += 1

In [6]:
bulos

{0: {'titulo': 'Más de 300 bulos y desinformaciones sobre migrantes, refugiados y minorías religiosas que te están intentando colar en el Día Internacional del Migrante',
  'link': 'https://maldita.es/malditobulo/20211218/300-bulos-desinformaciones-dia-migrante/',
  'date': '18/12/2021',
  'words_count': {'mas': 56,
   'de': 2382,
   '300': 4,
   'bulos': 14,
   'y': 767,
   'desinformaciones': 6,
   'sobre': 55,
   'migrantes': 23,
   'refugiados': 24,
   'minorias': 2,
   'religiosas': 1,
   'que': 1610,
   'te': 11,
   'estan': 43,
   'intentando': 5,
   'colar': 1,
   'en': 1442,
   'el': 1004,
   'dia': 22,
   'internacional': 5,
   'del': 286,
   'migrante': 5,
   'este': 173,
   '18': 22,
   'diciembre': 9,
   'es': 505,
   'un': 856,
   'colectivo': 2,
   'al': 154,
   '2020': 21,
   'pertenecian': 2,
   'unos': 22,
   '281': 1,
   'millones': 13,
   'personas': 118,
   'mundo': 14,
   'segun': 51,
   'la': 1119,
   'onu': 1,
   'diariamente': 1,
   'se': 680,
   'difunden': 8,

In [7]:
len(bulos)

10

In [ ]:
with open('fakenewsMaldita.json', 'w') as fp:
    json.dump(bulos, fp)